# Readable map

## Install packages
Best is to start a new environment, assuming your using [anaconda](https://www.anaconda.com/) see [anaconda documentation](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html)

basic ones included with most python

In [1]:
import base64
import os
from time import sleep
import io
from pathlib import Path
import requests

probably need to add a few, you can uncomment the ones needed

In [2]:
# data management and reading files
import pandas as pd
import openpyxl

# geopackages
import folium
from geopy.geocoders import Nominatim

In [3]:
# pip install pandas openpyxl

In [4]:
# pip install folium geopy

In [5]:
import reportlab

In [6]:
# pdf editios
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from PyPDF2 import PdfWriter, PdfReader

In [7]:
# pip install reportlab PyPDF2

In [8]:
# screenshot/chrome editor
from selenium import webdriver
from selenium.webdriver.common.print_page_options import PrintOptions

In [9]:
# pip install selenium

If Selenium doesn't work with `pip install selenium`, it requires some more setup, see [the docs for info](https://selenium-python.readthedocs.io/installation.html), you need the correct drivers, e.g.:
[Chrome](https://sites.google.com/chromium.org/driver/)
[Edge](https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/)
[Firefox](https://github.com/mozilla/geckodriver/releases)
[Safari](https://webkit.org/blog/6900/webdriver-support-in-safari-10/)

Also try reboot or a new environment

I used the versions listen below. newer versions may cause issues

In [17]:
import selenium, PyPDF2, reportlab

In [20]:
selenium.__version__, PyPDF2.__version__, reportlab.__version__, folium.__version__

('4.31.1', '3.0.1', '4.2.2', '0.17.0')

# Run code

In [22]:
# Lees de tabel met adressen (bijvoorbeeld van een CSV-bestand)
# data_csv = pd.read_excel("adressen.csv", delimiter=";")  # Zorg ervoor dat de CSV-kolommen juist zijn benoemd
data_csv = pd.read_excel("adressen.xlsx").

# Geocoder initialiseren
geolocator = Nominatim(user_agent="my_geocoder")

def get_osm_route(origin, inter_1, inter_2, inter_3, end):
    # url = f"https://map.project-osrm.org/?z=11&center={origin[0]}%2C{origin[1]}&loc={origin[0]}%2C{origin[1]}&loc={inter_1[0]}%2C{inter_1[1]}&loc={inter_2[0]}%2C{inter_2[1]}&loc={inter_3[0]}%2C{inter_3[1]}&loc={end[0]}%2C{end[1]}&hl=en&alt=0&srv=1"
    url = f"http://router.project-osrm.org/route/v1/bike/{origin[1]},{origin[0]};{origin[1]},{origin[0]};{inter_1[1]},{inter_1[0]};{inter_2[1]},{inter_2[0]};{inter_3[1]},{inter_3[0]};{end[1]},{end[0]}?overview=full&geometries=geojson"
    response = requests.get(url)
    # return response
    data = response.json()
    if 'routes' in data and len(data['routes']) > 0:
        geometry = data['routes'][0]['geometry']['coordinates']
        route = [(point[1], point[0]) for point in geometry]
        return route
    else:
        return []
    return url

def bounding_box(points):
    x_coordinates, y_coordinates = zip(*points)

    return [(min(x_coordinates), min(y_coordinates)), (max(x_coordinates), max(y_coordinates))]
    
# Loop door elke rij in de tabel
for index, row in data_csv.iterrows():
    # Haal de adressen op uit de huidige rij
    addresses = [
        row['Adres1'],
        row['Adres2'],
        row['Adres3'],
        row['Adres4'],
        row['Adres5']
    ]
    # Coördinaten ophalen met geocoding
    coordinates = []
    for addr in addresses:
        location = geolocator.geocode(addr)
        if location:
            coordinates.append((location.latitude, location.longitude))
        else:
            print(f'Ik kon dit adres: {addr} niet vinden, controleer de spelling!')
    # Kaart initialiseren
    center_lat = sum(coord[0] for coord in coordinates) / len(coordinates)
    center_lng = sum(coord[1] for coord in coordinates) / len(coordinates)
    m = folium.Map(location=[center_lat, center_lng], zoom_start=15.8, zoom_control=False, scrollWheelZoom=False,dragging=False)
    ### evnt andere kaart type
    # folium.TileLayer('cartodbdark_matter').add_to(m)
    # print(coordinates,addresses)
    # Markeringen toevoegen voor de adressen
    for i, addr in enumerate(addresses):
        folium.Marker(coordinates[i], popup=addr).add_to(m)
        if i == len(addresses)-1:
            pass
        else:
            folium.Marker(coordinates[i],icon=folium.DivIcon(html=f"""<div style='color: blue; fill= white'>{i+1}:{addr.split(",")[0]}</div>""")).add_to(m)
    
    # Fietsroute PolyLine toevoegen
    route_coords = coordinates
    route = get_osm_route(route_coords[0], route_coords[1], route_coords[2], route_coords[3], route_coords[-1])
    
    # folium.PolyLine(route, color="blue").add_to(m)
    m.fit_bounds(bounding_box(route), max_zoom=24)  # Zoom to route map

    # Opslaan als HTML-bestand
    path = Path.cwd()
    ouput_path = path / 'output'
    ouput_path.mkdir(exist_ok=True)
    
    html_file = ouput_path / f"fietsroutes_{index+1}.html"
    screenshot_file = ouput_path / f"Route_{index+1}.pdf"
    m.save(html_file)

    htmlpath = html_file 
    browser = webdriver.Chrome()
    browser.get(str(htmlpath))

    print("Waiting till map is loaded...")
    sleep(2)

    print_options = PrintOptions()
    print_options.margin_top = 0.0
    print_options.margin_bottom = 0.0
    print_options.margin_left = 0.0
    print_options.margin_right = 0.0
    print_options.shrink_to_fit = True
    print_options.orientation = "landscape"

    pdf_as_base64 = browser.print_page(print_options=print_options)
    convertedbytes = base64.b64decode(pdf_as_base64)
    with open(screenshot_file, 'wb') as pdf:
        pdf.write(convertedbytes)

    # adjust text accordingly
    tijden = ["","(Aankomst:18.00)","(Aankomst:18.45)","(Aankomst:19.30)","(Aankomst:20.15)"]
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=A4)
    split_n = 3
    can.setFillColorRGB(0.827,0.827, 0.827)
    can.rect(16,7,550,50, fill=1)
    can.rect(9,572,15,15, fill=1)
    can.setFillColorRGB(0,0,0)
    can.drawString(10, 575, f'{index+1}')
    can.drawString(20, 10, "Houd te allen tijde je mouwtjeshemd SCHOON en wees op tijd")
    can.drawString(20, 40, ", ".join([f'{i+1}:{addr.split(",")[0]}{tijden[i]}' for i, addr in enumerate(addresses[:split_n])]))
    can.drawString(20, 25, ", ".join([f'{i+1+split_n}:{addr.split(",")[0]}{tijden[i+split_n]}' for i, addr in enumerate(addresses[split_n:])]))

    
    can.save()
    
    #move to the beginning of the StringIO buffer
    packet.seek(0)
    
    # create a new PDF with Reportlab
    new_pdf = PdfReader(packet)
    # read your existing PDF
    existing_pdf = PdfReader(open(screenshot_file, "rb"))
    output = PdfWriter()
    # add the "watermark" (which is the new pdf) on the existing page
    page = existing_pdf.pages[0]
    page.merge_page(new_pdf.pages[0])
    output.add_page(page)
    # finally, write "output" to a real file
    output_stream = open(screenshot_file, "wb")
    output.write(output_stream)
    output_stream.close()
    print("Saved")
    try:
        browser.close()
    except:
        print('failed to verify if browser is closed')
        del browser


Waiting till map is loaded...
Saved
failed to verify if browser is closed
Waiting till map is loaded...
Saved
Waiting till map is loaded...
Saved
Waiting till map is loaded...
Saved
failed to verify if browser is closed
